In [15]:
#import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.graphics.tsaplots import plot_acf
import os

In [16]:
def generate_data(T, alpha, beta, sigma, alpha_y, beta_y, gamma, theta, sigma_y, mu):
    df = pd.DataFrame(index=range(T), columns=['x', 'y', 'epsilon_x', 'epsilon_y'])
    x1 = alpha/(1-beta) # define initial value for time series x
    y1 = 0
    epsilon_x = sigma*np.random.randn(T) + mu # generate a vector of T random normal
    epsilon_y = sigma_y*np.random.randn(T) # generate a vector of T random normal
    
    df.iloc[0,:] = [x1, y1, np.nan, np.nan] # initialize x and y
    
    x = np.zeros(T)
    y = np.zeros(T)
    x[0] = x1
    y[0] = y1
    
    for i in range(0, T-1):
        x[i+1] = alpha + beta * x[i] + epsilon_x[i+1] # generate X(t) recursively
            # Xt = α + βXt-1 + εt
        y[i+1] = alpha_y + beta_y * y[i] + gamma * x[i+1] + epsilon_y[i+1] + theta * epsilon_y[i] # generate Y(t) recursively
            # Yt = α + βYt-1 + γXt + εt + θεt-1
        df.iloc[i+1,0] = x[i+1]
        df.iloc[i+1,1] = y[i+1]
        df.iloc[i+1,2] = epsilon_x[i+1] # associate error with X(t)
        df.iloc[i+1,3] = epsilon_y[i+1] # associate error with Y(t)
    
    return df, x, y, epsilon_x, epsilon_y

In [17]:
df, x, y, epsilon_x, epsilon_y = generate_data(1000, 1, 0.9, 0.1, 1, 0.9, 0.9, 0.5, 0.1, 0)
new_df = df.drop(0)

In [37]:
import numpy as np
from scipy.stats import norm

def quantile_regression_gls(y, X, tau):
    n, p = X.shape
    nq = len(tau)
    residuals = np.zeros((n, nq))
    beta = np.zeros((nq, p))
    weights = np.zeros((n, n))
    Xw = np.zeros((n, nq*p+1))

    for i in range(nq):
        q = tau[i]
        for j in range(n):
            residuals[j, i] = y[j] - np.dot(X[j,:], beta[i,:])
            weights[j,j] = norm.pdf(residuals[j,i]/weights[j,j])
        Xw[:, i*p+1:(i+1)*p+1] = np.multiply(X, np.tile(weights[:,i],(p,1)).T)
        #Xw[:, i*p] = np.tile(weights[:,i],(1,1)).T
        Xw[:, i*p+1:(i+1)*p+1] = np.multiply(X, np.tile(weights[:,i, np.newaxis],(1,p)))

        #debugging
        rank = np.linalg.matrix_rank(Xw.T @ Xw)
        #beta[i,:] = np.linalg.inv(Xw.T @ Xw) @ Xw.T @ np.multiply(y, np.tile(weights[:,i],(1,1)).T)

    #return beta
    return rank


In [33]:
print(X.shape)
print(beta.shape)
print(y.shape)

(1000, 4)
(3, 3)
(1000,)


In [38]:
y_lag = np.roll(y, 1)
y_lag[0] = 0
epsilon_lag = np.roll(epsilon_y, 1)
epsilon_lag[0] = 0
X = np.column_stack((np.ones_like(x), y_lag, x, epsilon_lag)) #question
tau = [0.1, 0.5, 0.9]
quantile_regression_gls(y, X, tau)

/var/folders/f2/mjfy7nsn0tqbsmsk74mmlybh0000gn/T/ipykernel_24253/3250660834.py:16: RuntimeWarning: invalid value encountered in double_scalars
  weights[j,j] = norm.pdf(residuals[j,i]/weights[j,j])
/var/folders/f2/mjfy7nsn0tqbsmsk74mmlybh0000gn/T/ipykernel_24253/3250660834.py:16: RuntimeWarning: divide by zero encountered in double_scalars
  weights[j,j] = norm.pdf(residuals[j,i]/weights[j,j])


LinAlgError: SVD did not converge

In [36]:
rank = np.linalg.matrix_rank(Xw.T @ Xw)


NameError: name 'Xw' is not defined